## 1. Importar Librerías

Importamos las bibliotecas necesarias incluyendo `Annotated` y `Field` de Pydantic para definir herramientas.

In [ ]:
import asyncio
import random
from typing import Annotated
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from azure.identity import DefaultAzureCredential
from azure.identity.aio import get_bearer_token_provider
from pydantic import Field
from dotenv import load_dotenv
import os

## 2. Cargar Variables de Entorno

In [ ]:
load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model = os.getenv("MODEL")

print(f"Endpoint: {endpoint}")
print(f"Modelo: {model}")

## 3. Configurar Cliente de OpenAI

In [ ]:
client = OpenAIChatClient(
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT") + "/openai/v1/",
    api_key=get_bearer_token_provider(
        DefaultAzureCredential(), 
        "https://cognitiveservices.azure.com/.default"
    ),
    model_id=os.getenv("MODEL")
)

## 4. Definir la Herramienta del Clima

Esta función simula una consulta de clima. Los parámetros usan `Annotated` con `Field` para proporcionar descripciones que el modelo LLM usará para entender cuándo y cómo llamar a la función.

**Nota**: Esta es una simulación. En producción, esto llamaría a una API real de clima.

In [ ]:
def get_weather(
    city: Annotated[str, Field(description="City name, spelled out fully")],
) -> dict:
    """Simula una consulta de clima para una ciudad."""
    print(f"Getting weather for {city}")
    
    # Simulación aleatoria
    if random.random() < 0.5:
        return {"temperature": 72, "description": "Sunny"}
    else:
        return {"temperature": 60, "description": "Rainy"}

print("Herramienta get_weather definida")

## 5. Crear el Agente con Herramienta

Creamos un agente y le pasamos la herramienta `get_weather`. El agente decidirá automáticamente cuándo usar esta herramienta basándose en la consulta del usuario.

In [ ]:
agent = ChatAgent(
    chat_client=client, 
    instructions="You're an informational agent. Answer questions cheerfully.", 
    tools=[get_weather]
)

print("Agente creado con herramienta de clima")

## 6. Ejecutar Consulta de Clima

Cuando preguntamos sobre el clima, el agente:
1. Reconoce que necesita información del clima
2. Llama automáticamente a la función `get_weather`
3. Usa el resultado para responder al usuario

In [ ]:
async def main():
    print("\n=== Consultando clima ===")
    response = await agent.run("Whats weather today in sf?")
    print(f"\nRespuesta del agente:\n{response}")

# Ejecutar en entorno Jupyter
await main()

## 7. Probar Consultas sin Herramienta

El agente también puede responder preguntas que no requieren herramientas.

In [ ]:
async def test_general_question():
    print("\n=== Pregunta general ===")
    response = await agent.run("What is the capital of France?")
    print(f"\nRespuesta del agente:\n{response}")

await test_general_question()

## 8. Múltiples Consultas de Clima

Podemos hacer varias consultas sobre diferentes ciudades.

In [ ]:
async def test_multiple_cities():
    cities = ["Tokyo", "London", "New York"]
    
    print("\n=== Consultando clima en múltiples ciudades ===")
    for city in cities:
        print(f"\n--- {city} ---")
        response = await agent.run(f"What's the weather like in {city}?")
        print(f"Respuesta: {response.text}")

await test_multiple_cities()

## Conclusión

Este ejemplo demuestra:
1. **Function Calling**: El modelo LLM puede llamar automáticamente a funciones Python
2. **Anotaciones de tipo**: Uso de `Annotated` y `Field` para documentar parámetros
3. **Decisión automática**: El agente decide cuándo usar herramientas
4. **Integración natural**: El resultado de la función se integra en la respuesta

Aplicaciones prácticas:
- Consulta de APIs externas (clima, noticias, stocks)
- Acceso a bases de datos
- Cálculos complejos
- Interacción con sistemas externos

El framework maneja automáticamente la serialización/deserialización de datos entre el modelo y las funciones.